In [19]:
import xmltodict
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
#from gensim import corpora, models, similarities 

time: 35 ms


In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 17 ms


In [21]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 773 ms


In [22]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 11 ms


[autoreload of autotime failed: Traceback (most recent call last):
  File "C:\Anaconda\envs\py27\lib\site-packages\IPython\extensions\autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named autotime
]


In [23]:
ls data

 Volume in drive E has no label.
 Volume Serial Number is 19C7-A2B3

 Directory of E:\metis\ds4_fletcher\data

08/17/2015  12:55 PM    <DIR>          .
08/17/2015  12:55 PM    <DIR>          ..
03/09/2015  06:04 PM        60,201,826 Badges.xml
03/09/2015  06:04 PM       238,716,194 Comments.xml
03/09/2015  06:05 PM     1,144,804,160 PostHistory.xml
03/09/2015  06:06 PM         6,192,845 PostLinks.xml
03/09/2015  06:06 PM       736,314,717 Posts.xml
03/09/2015  06:06 PM           367,147 Tags.xml
03/09/2015  06:06 PM       100,081,558 Users.xml
03/09/2015  06:07 PM       210,233,404 Votes.xml
               8 File(s)  2,496,911,851 bytes
               2 Dir(s)  638,569,029,632 bytes free
time: 236 ms


In [24]:
data_dir='data'

time: 5 ms


In [48]:

posts_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Posts.xml'),max_num_children=1000)

#posts_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Posts.xml'))

time: 1min 13s


In [49]:
posts_df.columns

Index([u'AcceptedAnswerId', u'AnswerCount', u'Body', u'ClosedDate',
       u'CommentCount', u'CommunityOwnedDate', u'CreationDate',
       u'FavoriteCount', u'Id', u'LastActivityDate', u'LastEditDate',
       u'LastEditorDisplayName', u'LastEditorUserId', u'OwnerDisplayName',
       u'OwnerUserId', u'ParentId', u'PostTypeId', u'Score', u'Tags', u'Title',
       u'ViewCount'],
      dtype='object')

time: 19 ms


In [73]:
posts_df.irow(0).Body

u'<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes t

time: 57 ms


In [50]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,NaN,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,NaN,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654
2,NaN,NaN,<p>You could use a filter on the recording. If...,NaN,0,NaN,2009-07-08T16:34:52.867,NaN,25,2009-07-15T06:48:44.300,...,NaN,NaN,Burkhard,9637,24,2,0,NaN,NaN,NaN
3,NaN,NaN,<p>try out something like:</p>\n\n<p>audible (...,NaN,0,NaN,2009-07-08T16:59:00.587,NaN,26,2009-07-15T06:48:44.300,...,NaN,NaN,Jas Panesar,23242,24,2,0,NaN,NaN,NaN
4,NaN,NaN,<p>You definitely need some sort of software t...,NaN,1,NaN,2009-07-08T17:15:59.477,NaN,27,2009-07-15T06:48:44.300,...,NaN,NaN,T Pops,45675,24,2,0,NaN,NaN,NaN


time: 491 ms


Only interested in questions

In [53]:
print("There are %s posts"%(len(posts_df)))
posts_df=posts_df[posts_df.PostTypeId=="1"]
print("There are %s question posts"%(len(posts_df)))

There are 1001 posts
There are 293 question posts
time: 5 ms


## Process Text

In [54]:
# strip html tags
posts_df['BodyCleaned']=posts_df['Body'].apply(lambda x : helper.strip_tags(x))
# tokenize text
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.tokenize_and_stem(x))

# remove stop words
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.remove_stopwords(x))



time: 8.33 s


In [55]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount,BodyCleaned
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584,"[vista, virtual, machin, use, onli, gb, virtua..."
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654,"[use, record, import, technic, meet, demo, use..."
5,42,9,<p>What is the difference between the two supe...,NaN,0,NaN,2009-07-15T07:11:46.370,1,29,2011-11-25T19:11:50.960,...,48078,NaN,13,NaN,1,17,<linux><unix><sudo>,"What's the difference between the commands ""su...",14700,"[differ, two, super, user, command, su, -s, su..."
7,3841,3,"<p>My late 2008, Macbook Pro Unibody machine h...",NaN,2,NaN,2009-07-15T07:17:13.970,1,37,2009-07-15T21:15:21.323,...,NaN,NaN,26,NaN,1,4,<mac><crash><boot-camp>,Why does the Macbook Pro Unibody crash on hibe...,1169,"[late, macbook, pro, unibodi, machin, number, ..."
13,92,2,<p>Is there any way to configure application t...,NaN,0,NaN,2009-07-15T07:20:00.717,1,46,2014-07-26T13:37:09.967,...,81403,NaN,21,NaN,1,9,<windows-vista><boot><uac><privileges><elevation>,Elevated Priviliges for Startup Applications i...,7997,"[ani, way, configur, applic, requir, administr..."


time: 71 ms


In [56]:
len(posts_df)

293

time: 3 ms


In [57]:
texts = posts_df['BodyCleaned']

time: 3 ms


In [69]:
?dictionary.filter_extremes

time: 202 ms


In [58]:
#create a Gensim dictionary from the texts
dictionary = gensim.corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

time: 116 ms


In [59]:
corpus

[[(16, 1),
  (56, 3),
  (65, 1),
  (86, 1),
  (89, 1),
  (95, 1),
  (115, 1),
  (119, 1),
  (131, 1),
  (143, 1),
  (144, 1),
  (145, 4),
  (159, 1),
  (170, 4),
  (191, 5),
  (198, 1),
  (199, 1),
  (211, 1),
  (212, 2),
  (247, 1),
  (252, 7),
  (259, 1),
  (285, 1),
  (302, 1),
  (330, 1),
  (353, 2),
  (360, 9),
  (392, 1),
  (418, 1),
  (436, 1),
  (437, 1),
  (451, 4),
  (458, 2),
  (507, 1),
  (513, 2),
  (518, 2),
  (525, 1),
  (526, 1),
  (570, 1),
  (617, 1),
  (640, 1),
  (649, 1),
  (651, 3),
  (659, 1),
  (670, 2),
  (674, 1),
  (679, 2),
  (694, 1),
  (713, 1),
  (725, 2),
  (729, 1),
  (763, 2),
  (772, 1),
  (774, 1),
  (788, 1),
  (798, 1),
  (800, 1),
  (826, 1),
  (837, 1),
  (863, 7),
  (874, 1),
  (890, 1),
  (899, 4),
  (915, 2),
  (932, 1),
  (933, 1),
  (937, 2),
  (939, 1),
  (945, 2),
  (953, 1),
  (966, 1),
  (996, 1),
  (1007, 1),
  (1010, 1),
  (1036, 4),
  (1047, 3),
  (1094, 1),
  (1099, 1),
  (1113, 1),
  (1132, 1),
  (1135, 1),
  (1142, 1),
  (1162, 1),

time: 1.28 s


In [64]:
#gensim.models.LdaMulticore
lda = gensim.models.LdaModel(corpus, num_topics=10, passes=10,
                            id2word=dictionary)

time: 14.4 s


In [67]:
lda.show_topics(num_words=20)


[u"0.013*n't + 0.013*window + 0.013*ani + 0.012*doe + 0.011*use + 0.010*'s + 0.009*sync + 0.009*get + 0.008*'m + 0.008*way + 0.008*ve + 0.007*work + 0.007*instal + 0.007*comput + 0.007*googl + 0.006*like + 0.006*updat + 0.006*contact + 0.006*card + 0.005*usb",
 u"0.020*drive + 0.014*like + 0.014*window + 0.013*use + 0.010*machin + 0.010*hard + 0.009*n't + 0.009*key + 0.008*ve + 0.008*want + 0.007*'d + 0.007*mac + 0.007*get + 0.007*pc + 0.007*would + 0.007*file + 0.006*way + 0.006*know + 0.006*work + 0.006*raid",
 u"0.023*use + 0.018*window + 0.011*work + 0.010*connect + 0.009*would + 0.009*instal + 0.009*'s + 0.008*differ + 0.007*'m + 0.007*one + 0.007*network + 0.007*ani + 0.006*set + 0.006*vista + 0.006*file + 0.006*comput + 0.006*video + 0.006*ve + 0.006*onli + 0.006*linux",
 u"0.014*'s + 0.012*comput + 0.011*use + 0.010*like + 0.010*fan + 0.010*monitor + 0.010*get + 0.007*doe + 0.007*also + 0.007*n't + 0.006*chang + 0.006*dust + 0.005*know + 0.005*speed + 0.005*browser + 0.005*set 

time: 9 ms


In [66]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

["n't", 'window', 'ani', 'doe', 'use', "'s", 'sync', 'get', "'m", 'way', 've', 'work', 'instal', 'comput', 'googl', 'like', 'updat', 'contact', 'card', 'usb']
()
['drive', 'like', 'window', 'use', 'machin', 'hard', "n't", 'key', 've', 'want', "'d", 'mac', 'get', 'pc', 'would', 'file', 'way', 'know', 'work', 'raid']
()
['use', 'window', 'work', 'connect', 'would', 'instal', "'s", 'differ', "'m", 'one', 'network', 'ani', 'set', 'vista', 'file', 'comput', 'video', 've', 'onli', 'linux']
()
["'s", 'comput', 'use', 'like', 'fan', 'monitor', 'get', 'doe', 'also', "n't", 'chang', 'dust', 'know', 'speed', 'browser', 'set', 'stand', 'seem', 've', 'anyon']
()
['use', 'would', 'window', 'like', 'work', 'get', 'devic', 'want', 'ani', 'error', "n't", 'router', 'set', 'comput', 'bar', 'pci', 'alloc', 'mem', 'play', 'wireless']
()
['flash', 'color', 'cooki', 'top', 'name', 'use', 'folder', 'delet', 'sharedobject', 'creat', 'allow', 'drive', 'third-parti', 'browser', 'domain', 'set', 'even', 'pc', 'te